# <center> Project: Visualizing Real World Data  </center>

##  <center> Health decline in post-Soviet Union countries in relation to nutrition, smoking and alcohol consumption </center>

###  <center> Data cleaning </center>

###### <center> Kamila Kutz-Cerri, Ironhack Data Analytics Bootcamp, Berlin, Autumn 2019 </center>

#### Importing modules

In [1]:
import pandas as pd
import numpy as np

#####   Loading the data 
source: http://www.euro.who.int/en/data-and-evidence/archive/european-health-for-all-database-hfa-db

In [2]:
hfa_1 = pd.read_csv('HFA Data (table) part 1.csv')
hfa_2 = pd.read_csv('HFA Data (table) part 2.csv')
hfa_3 = pd.read_csv('HFA Data (table) part 3.csv')

/usr/local/Cellar/ipython/7.8.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
hfa_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499936 entries, 0 to 499935
Data columns (total 5 columns):
Measure code      499936 non-null object
SEX               499936 non-null object
COUNTRY_REGION    499936 non-null object
YEAR              499936 non-null int64
VALUE             499936 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 19.1+ MB


In [4]:
hfa_1['Measure code'].describe()

count     499936
unique       168
top       HFA_19
freq        8749
Name: Measure code, dtype: object

- The column 'Measure code' refers to a specific parameter that was was measured but there is no description of the parameter. It's necessary to load another dataset

In [5]:
df_label = pd.read_excel('HFA Metadata.xlsx', sheet_name='Labels', header =1)

In [6]:
df_label

,Code,Label,Unnamed: 2
0,HFA_1,"Mid-year population, by sex",NaN
1,HFA_2,Mid-year male population,NaN
2,HFA_3,Mid-year female population,NaN
3,HFA_10,"% of population aged 0–14 years, by sex",NaN
4,HFA_11,"% of population aged 0–14 years, males",NaN
...,...,...,...
703,YES_NO_UNK,YES,Yes
704,YES_NO_UNK,NO,No
705,YES_NO_UNK,UNK,Unknown
706,YES_NO_UNK,NORESP,No response


##### Setting the 'Code' column as an index

In [7]:
df_label = df_label.set_index('Code')

In [8]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
Index: 708 entries, HFA_1 to YES_NO_UNK
Data columns (total 2 columns):
Label         694 non-null object
Unnamed: 2    19 non-null object
dtypes: object(2)
memory usage: 16.6+ KB


### Merge all csvs into one dataframe

In [9]:
df = hfa_1.append(hfa_2, sort=False).append(hfa_3, sort=False)

In [10]:
df.columns

Index(['Measure code', 'SEX', 'COUNTRY_REGION', 'YEAR', 'VALUE',
       'PLACE_RESIDENCE', 'YES_NO'],
      dtype='object')

### Add description of the 'Measure code'

In [11]:
vals_to_replace = df_label['Label'].to_dict()
df['description'] = df['Measure code'].map(vals_to_replace)

In [12]:
df.head()

,Measure code,SEX,COUNTRY_REGION,YEAR,VALUE,PLACE_RESIDENCE,YES_NO,description
0,HFA_1,ALL,ALB,1970.0,2138000.0,NaN,NaN,"Mid-year population, by sex"
1,HFA_1,ALL,ALB,1971.0,2195600.0,NaN,NaN,"Mid-year population, by sex"
2,HFA_1,ALL,ALB,1972.0,2253200.0,NaN,NaN,"Mid-year population, by sex"
3,HFA_1,ALL,ALB,1973.0,2310800.0,NaN,NaN,"Mid-year population, by sex"
4,HFA_1,ALL,ALB,1974.0,2368400.0,NaN,NaN,"Mid-year population, by sex"


### Clean column names

In [13]:
df.columns = df.columns.str.replace('Measure' ,'').str.replace('_REGION', '').str.strip()
df.columns = df.columns.str.replace('_',' ').str.lower()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1374762 entries, 0 to 377457
Data columns (total 8 columns):
code               1374762 non-null object
sex                1374709 non-null object
country            1374762 non-null object
year               1374709 non-null float64
value              1374762 non-null float64
place residence    9208 non-null object
yes no             53 non-null object
description        1374762 non-null object
dtypes: float64(2), object(6)
memory usage: 94.4+ MB


- Some columns have many missing values. Checking which 'measure codes' have information about 'yes no' and 'place residence'

In [18]:
(df.loc[~df['yes no'].isna()]['code'].unique(),
 df.loc[~df['yes no'].isna()]['description'].unique())

(array(['HFA_633'], dtype=object),
 array(['WHO Member States participating in the European Health Information Initiative (EHII)'],
       dtype=object))

In [19]:
(df.loc[~df['place residence'].isna()]['code'].unique(),
 df.loc[~df['place residence'].isna()]['description'].unique())

(array(['HFA_463', 'HFA_466'], dtype=object),
 array(['% population with homes connected to water supply system, by place of residence',
        '% population with access to sewage system, septic tank or other hygienic means of sewage disposal, by place of residence'],
       dtype=object))

- 'HFA_633', 'HFA_463' and 'HFA_466' contain information not important for this analaysis. They will be excluded from the dataframe

In [21]:
df = df.loc[~df['code'].isin(['HFA_463', 'HFA_466', 'HFA_633'])]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365501 entries, 0 to 377404
Data columns (total 8 columns):
code               1365501 non-null object
sex                1365501 non-null object
country            1365501 non-null object
year               1365501 non-null float64
value              1365501 non-null float64
place residence    0 non-null object
yes no             0 non-null object
description        1365501 non-null object
dtypes: float64(2), object(6)
memory usage: 93.8+ MB


### Deleting columns with NaNs: 'place residence' and 'yes no'

In [23]:
df = df.dropna(axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365501 entries, 0 to 377404
Data columns (total 6 columns):
code           1365501 non-null object
sex            1365501 non-null object
country        1365501 non-null object
year           1365501 non-null float64
value          1365501 non-null float64
description    1365501 non-null object
dtypes: float64(2), object(4)
memory usage: 72.9+ MB


### Checking if there are any duplicated columns

In [24]:
df.duplicated().value_counts()

False    1365501
dtype: int64

### Convert 'year' into float

In [25]:
df['year'] = df['year'].astype(int)

### Check country names and country groups

In [27]:
df['country'].unique()

array(['ALB', 'ISL', 'BGR', 'GEO', 'IRL', 'ARM', 'AUT', 'MLT', 'MCO',
       'GBR', 'MKD', 'HUN', 'FIN', 'AND', 'UKR', 'BEL', 'AZE', 'BIH',
       'BLR', 'WHO_EURO', 'CZE', 'EST', 'FRA', 'GRC', 'ISR', 'KAZ', 'ITA',
       'UZB', 'NORDIC', 'HRV', 'NOR', 'LUX', 'DNK', 'ROU', 'DEU', 'ESP',
       'LVA', 'POL', 'MNE', 'PRT', 'RUS', 'SVK', 'SVN', 'TJK', 'KGZ',
       'CIS', 'CHE', 'LTU', 'NLD', 'TKM', 'SWE', 'CYP', 'EU_MEMBERS',
       'SMR', 'MDA', 'EU_AFTER_MAY2004', 'CARINFONET', 'SRB',
       'EU_BEFORE_MAY2004', 'SEEHN', 'TUR', 'SMALL'], dtype=object)

- There are some country groups, i will exclude them from analysis

In [29]:
df = df.loc[~df['country'].isin([
    'WHO_EURO',
    'EU_MEMBERS',
    'EU_BEFORE_MAY2004',
    'EU_AFTER_MAY2004',
    'CIS',
    'CARINFONET',
    'SEEHN',
    'NORDIC',
    'SMALL'
])]

In [30]:
df.head()

,code,sex,country,year,value,description
0,HFA_1,ALL,ALB,1970,2138000.0,"Mid-year population, by sex"
1,HFA_1,ALL,ALB,1971,2195600.0,"Mid-year population, by sex"
2,HFA_1,ALL,ALB,1972,2253200.0,"Mid-year population, by sex"
3,HFA_1,ALL,ALB,1973,2310800.0,"Mid-year population, by sex"
4,HFA_1,ALL,ALB,1974,2368400.0,"Mid-year population, by sex"


 ### Post-soviet countries (labeled as 'east' for the purpose of the analysis) 
- Armenia: 'ARM'
- Azerbaijan: 'AZE'
- Belarus: 'BLR'
- Estonia: 'EST'
- Georgia: 'GEO'
- Kazakhstan: 'KAZ'
- Kyrgyzstan: 'KGZ'
- Latvia: 'LTV'
- Lithuania: 'LTU'
- Moldova: 'MDA'
- Russia: 'RUS'
- Tajikistan: 'TJK'
- Turkmenistan: 'TKM' 
- Ukraine: 'UKR'
- Uzbekistan: 'UZB'
- Poland: 'POL' (Poland was not part of the Soviet Union, but it was considered a Soviet “satellite state” with a Communist government friendly to Moscow who was the real power)

In [31]:
east = ['RUS','EST','KAZ','KGZ','MDA','LTV','BLR','UKR','POL','UZB','TKM','AZE','TJK','LTU','ARM' ,'GEO']

#### Selecting a time span for the analysis: 1985 to 2014

In [32]:
df = df[(df['year'] >= 1985)
               & (df['year'] <= 2014)]

### Adding a column to distinguish east (post-Soviet Union countries and Poland) from the rest (west

In [34]:
df['east west'] = np.where(df['country'].isin(east), 'east', 'west')

### Save the clean df as a csv file for the analysis and visualization

In [35]:
df.to_csv('hfa_clean.csv' ,index=False)